In [ ]:
using Polyhedra
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()
using RigidBodyDynamics
using StaticArrays
using RigidBodyTreeInspector

In [ ]:
vis = Visualizer()[:hopper]

In [ ]:
using JuMP, Cbc

In [ ]:
include("examples/articulated_complementarity.jl")

In [ ]:
g = -9.81
world = RigidBody{Float64}("world")
hopper = Mechanism(world; gravity=SVector(0, 0, g))

frame = CartesianFrame3D("core")
mass = 1.
inertia = SpatialInertia(frame, 0.01 * eye(SMatrix{3, 3}), SVector(0., 0, 0), mass)
core = RigidBody(inertia)

floating_base = Joint("core_to_world", QuaternionFloating{Float64}())
attach!(hopper, world, floating_base, eye(Transform3D, frame_before(floating_base), default_frame(world)), core)

frame = CartesianFrame3D("foot")
mass = 1.
inertia = SpatialInertia(frame, 0.01 * eye(SMatrix{3, 3}), SVector(0., 0, 0), mass)
foot = RigidBody(inertia)
leg = Joint("leg", Prismatic(SVector(0, 0, -1.)))
attach!(hopper, core, leg, eye(Transform3D, frame_before(leg), default_frame(core)), foot)

foot_contact = Point3D(default_frame(foot), SVector(0, 0, 0))

setgeometry!(vis, hopper, create_geometry(hopper, show_inertias=true))

In [ ]:
env = Complementarity.Environment(
    Dict(foot => Complementarity.ContactEnvironment(
            [foot_contact],
            [
                Complementarity.Obstacle(
                    SimpleHRepresentation{3, Float64}([0 0 1], [0]),
                    HalfSpace{3, Float64}([0, 0, 1], 0),
                    0.5
                ),
            ],
            [
                SimpleHRepresentation{3, Float64}(
                    [0 0 -1],
                    [0])
            ])
        )
)

limits = Dict(
    leg => SimpleHRepresentation{1, Float64}([1 -1]', [1.5, -0.5])
)

In [ ]:
x0 = MechanismState(hopper, [0., 0, 0, 1, 0, 0, 1.5, 1.0], zeros(num_velocities(hopper)))

u = zeros(num_velocities(x0))

model = Model(solver=CbcSolver())
up = Complementarity.update(x0, u, limits, env, 0.1, model)

In [ ]:
controller = (x) -> begin
    kp = 40
    kd = 0.1 * kp
    u = zeros(num_velocities(x))
    u[end] = kp * (1 - configuration(x)[end]) - kd * velocity(x)[end]
    u
end

results = Complementarity.simulate(x0, controller, limits, env, 0.05, 20);

In [ ]:
for r in results
    settransform!(vis, r.state)
    sleep(0.1)
end